<a href="https://colab.research.google.com/github/kankkw/229352-StatisticalLearning/blob/main/Lab10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

ปฏิบัติการครั้งที่ 10 กระบวนวิชา 229351 Statistical Learning for Data Science

คำชี้แจง

1. ให้เริ่มทำปฏิบัติการจาก colab notebook/ipynb ที่กำหนดให้ จากนั้น share แล้วส่ง link ใน Mango

ดาวน์โหลดข้อมูลผู้รับวินิจฉัยโรคหัวใจใน link ข้างล่างนี้  
https://donlapark.pages.dev/229351/data/framingham.csv

In [ ]:
# uploading the csv file to colab

!wget -O framingham.csv https://donlapark.pages.dev/229351/data/framingham.csv

In [ ]:
# import module ที่ต้องใช้
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
data = pd.read_csv('framingham.csv')

#drop rows with missing values
data = data.dropna()

print(data.info())

data.head()

### ตัวแปรตามคือ `TenYearCHD` มีค่าเป็น 1 เมื่อผู้มาใช้บริการมีโรคหัวใจภายใน 10 ปีหลังจากการตรวจสุขภาพ

### แบ่งเป็นตัวแปรต้น `X` กับตัวแปรตาม `y`

In [ ]:
y = data['TenYearCHD']

X = data.drop(['TenYearCHD'], axis = 1)

### สร้าง Logistic regression

$$ p(y=1|x_1,\ldots,x_p) = \frac{1}{1+e^{-(a_0+a_1x_1+\ldots+a_px_p)}} $$

### วิธีที่ 1: ใช้ `scikit-learn` ([Documentation](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html))

In [ ]:
from sklearn.linear_model import LogisticRegression

# สร้าง model object ก่อน
# ตั้ง random state ให้เป็น 0 เพื่อให้ผลลัพธ์ของทุกคนเหมือนกัน
model = LogisticRegression()

# แล้วค่อยใส่ข้อมูลโดยใช้ `fit`
model.fit(X,y)

### Warning บอกว่ามีปํญหาในการ optimize เนื่องจากบางค่าใน `X` มีขนาดใหญ่เกินไป (ตัวอย่างเช่นตัวแปร `totChol` มีค่า `>200`)

### เป็นตัวอย่างหนึ่งที่แสดงให้เห็นถึงความสำคัญของ standardization (ปรับให้ mean ของแต่ละตัวแปรเป็น 0 และ s.d. เป็น 1)  

### สมมติให้ `X1` เป็นตัวแปรหนึ่งใน dataset  

### Standardization ทำโดยการแปลง `X1 -> (X1 - mean(X1))/(s.d.(X1))`

In [ ]:
X.std()

In [ ]:
# Standardization
X_s = (X-X.mean())/X.std()

X_s

### ใช้ `X_s` แทน `X` แล้ว warning จะหายไป

In [ ]:
# warning จะหายไป
model.fit(X_s,y)

In [ ]:
# ดู intercept (a_0) ที่ได้โดยใช้
print('intercept:',model.intercept_)

# ดู coefficient ที่เหลือทั้งหมด (a_1,a_2,...,a_p) โดยใช้
print('coefficients:',model.coef_)

### ดูผลจากการใช้โมเดลแบ่งกลุ่ม `X_s` โดยใช้ `model.predict(X_s)`

In [ ]:
y_pred = model.predict(X_s)

print(y_pred)

### โมเดลแบ่งให้ข้อมูลส่วนใหญ่อยู่ในกลุ่มที่ 0 ทำให้เราไม่มั่นใจว่ามีข้อมูลที่ถูกแบ่งให้อยู่ในกลุ่มที่ 1 หรือไม่

### สามารถนับจำนวนข้อมูลที่ถูกแบ่งให้อยู่ในกลุ่มที่ 1 โดยคำนวณผลบวกของตัวเลขทั้งหมดที่อยู่ใน `y_pred`

In [ ]:
# คำนวณผลบวกของตัวเลขทั้งหมดที่อยู่ใน y_pred

print(y_pred.sum())

In [ ]:
y

### เราสามารถสร้าง array ที่ตรวจสอบว่าค่าที่ model ทำนาย (`y_pred`) ตรงกับค่าที่แท้จริง (`y`) หรือไม่โดยใช้ `y==y_pred`

| y | y_pred |       | y == y_pred |
|:------:|:------:|:-----:|:----------------:|
|    1   |    1   | $\to$ |         1        |
|    0   |    0   | $\to$ |         1        |
|    0   |    1   | $\to$ |         0        |
|    1   |    0   | $\to$ |         0        |

In [ ]:
# จะเห็นว่าโมเดลแบ่งกลุ่มให้ข้อมูลชิ้นที่ 3 ไม่ตรงกับกลุ่มที่แท้จริง

y == y_pred

In [ ]:
True + False

### จากตรงนี้ เราสามารถนับจำนวนข้อมูลที่โมเดลแบ่งกลุ่มได้ถูกต้องโดยใช้ `(y == y_pred).sum()` (เพราะว่าในการบวกค่า boolean นั้น `True`=0 และ `False`=1)

In [ ]:
(y == y_pred).sum()

## Exercise 1:

### ให้ `y_true` เป็นค่าที่แท้จริงทั้งหมด และ `y_pred` เป็นค่าที่โมเดลทำนายทั้งหมด ค่า accuracy คำนวณได้จาก

$$\text{Accuracy} = \frac{\# (\texttt{y_true} = \texttt{y_pred})}{N}$$

### โดยที่ $N$ คือจำนวนข้อมูลทั้งหมด

### จงสร้างฟังก์ชันที่คำนวณค่า Accuracy โดยมี inputs เป็น `y_true` และ `y_pred`

### Hint: ใช้ `y_true.shape[0]` ในการนับจำนวนข้อมูลทั้งหมด

In [ ]:
def accuracy(y_true, y_pred):
  '''
  y_true: numpy array of true labels
  y_pred: numpy array of model's classifications
  Return: the accuracy of the model
  '''
  y_true = np.asarray(y_true).ravel()
  y_pred = np.asarray(y_pred).ravel()
  return np.mean(y_true == y_pred)

# ทดสอบฟังก์ชัน accuracy
print(accuracy(np.array([1,0,0,0]),np.array([0,1,1,1]))) #ควรจะได้ 0
print(accuracy(np.array([1,0,0,0]),np.array([1,0,0,0]))) #ควรจะได้ 1
print(accuracy(np.array([1,1,0,0]),np.array([1,0,1,1]))) #ควรจะได้ 0.25

## Exercise 2:

จงคำนวณ accuracy ของโมเดล logistic regression ที่ได้ข้างบน

In [ ]:
acc_sklearn = accuracy(y, y_pred)
print("Accuracy (LogisticRegression, sklearn):", acc_sklearn)

### วิธีที่ 2: ใช้ `Logit` จาก `statsmodels` ([Documentation](https://www.statsmodels.org/stable/generated/statsmodels.discrete.discrete_model.Logit.html))

### ดูค่า Z-value และ p-value ของแต่ละตัวแปร

In [ ]:
from statsmodels.discrete.discrete_model import Logit

# โมเดลชื่อ logit
logit = Logit(y, X_s)

# จะเห็นว่า syntax ของ statsmodels นั้นใส่ข้อมูลก่อนแล้วค่อย fit
logit_fitted = logit.fit()

logit_fitted.summary()

## Exercise 3:

### จงระบุว่ามีตัวแปรไหนบ้างที่มีความสำคัญที่ระดับนัยสำคัญ 0.1

คำตอบ
ตัวแปรที่มีนัยสำคัญที่ระดับ 0.1 ได้แก่:

- male  
- age  
- cigsPerDay  
- prevalentHyp  
- totChol  
- sysBP  
- glucose

### ผลลัพธ์ที่ได้จากการใช้ `predict` จะแตกต่างจาก `scikit-learn` ตรงที่ output เป็น**ค่าความน่าจะเป็น** ไม่ใช่ค่าการแบ่งกลุ่ม

In [ ]:
logit_fitted.predict(X_s)

## Exercise 4:

ฟังก์ชันข้างล่างนี้มีหน้าที่เปลี่ยนค่าความน่าจะเป็นที่ได้ให้เป็นการแบ่งกลุ่ม (0 หรือ 1) จงเติมส่วนที่หายไปในฟังก์ชันข้างล่างนี้ให้สมบูรณ์

In [ ]:
def classify(y_pred):
  '''
  y_pred: numpy array of probabilities (จาก logit_fitted.predict)
  Return: list/array ของคลาสที่คาดการณ์ (0/1) โดยใช้ threshold = 0.5
  '''
  y_pred = np.asarray(y_pred).ravel()
  N = y_pred.shape[0] #number of observations
  results = [] #array of classifications

  for prob in y_pred:
    if prob >= 0.5:
      results.append(1)
    else:
      results.append(0)

  return results

#Test
print(classify(np.array([0.2,0.4,0.8]))) #should return [0,0,1]
print(classify(np.array([0.8,0.6,0.1]))) #should return [1,1,0]

## Exercise 5:

จงแสดงการแบ่งกลุ่ม (classification) ที่ได้จากโมเดล `logit_fitted` ในพื้นที่ข้างล่างนี้


In [ ]:
probs_stats = logit_fitted.predict(X_s)

y_pred_stats = np.array(classify(probs_stats))

print("First 20 class predictions:", y_pred_stats[:20])

acc_stats = accuracy(y, y_pred_stats)
print("Accuracy (Logit, statsmodels):", acc_stats)